This notebook contains the code to reproduce the results of the paper:

[B. Legat](https://perso.uclouvain.be/benoit.legat), [R. M. Jungers](https://perso.uclouvain.be/raphael.jungers/content/home). [**Parallel optimization on the Entropic Cone**](http://sites.uclouvain.be/sitb2016/Proceedings_SITB2016_preliminary.pdf). [*37rd Symposium on Information Theory in the Benelux*](http://sites.uclouvain.be/sitb2016), 2016.

In [ ]:
using EntropicCone

Select a solver installed. [`GLPK`](https://github.com/JuliaOpt/GLPK.jl) is not the fastest but it is free and reliable, [`Gurobi`](https://github.com/JuliaOpt/Gurobi.jl) is fast and reliable but it is commercial (although academic licenses are available). Many other solvers are available, see [here](http://www.juliaopt.org/) for a list.

In [ ]:
using GLPKMathProgInterface
solver = GLPKSolverLP()

In [ ]:
c = ingleton(4,1,2,3,4)

In [ ]:
cut = nonnegative(4, 1234)

In [ ]:
n = 4
h = polymatroidcone(Float64, 4)
newcut = :AddImmediately
(root, allnodes) = stochasticprogram(c, h, solver, 7, cut, newcut, [-1,-1,-1,-1,-1,-1,-1])